In [7]:
import pandas as pd
import boto3
from datetime import datetime
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler

In [8]:
data_noise = pd.read_csv('/Users/tianying/Documents/Statistics and Data Science/Course/Modern Data Analytics/MDA project/MDA-Georgia/Data(updated)/monthly_noisedata_2022.csv')
data_noise.head()

,month,description,lamax,laeq
0,1,MP 03: Naamsestraat 62 Taste,53.239009,51.727544
1,1,MP 05: Calvariekapel KU Leuven,50.374792,48.560197
2,1,MP 06: Parkstraat 2 La Filosovia,50.086348,48.274795
3,1,MP 07: Naamsestraat 81,48.800443,47.515371
4,2,MP 01: Naamsestraat 35 Maxim,57.287668,55.245502


In [9]:
# create a dataframe
gps_data = {
    'description': ['MP 01: Naamsestraat 35  Maxim', 'MP 02: Naamsestraat 57 Xior', 'MP 03: Naamsestraat 62 Taste', 'MP 04: His & Hears', 'MP 05: Calvariekapel KU Leuven', 'MP 06: Parkstraat 2 La Filosovia', 'MP 07: Naamsestraat 81', 'MP08bis - Vrijthof'],
    'lat': [50.877121, 50.87650, 50.87590, 50.875237, 50.87452, 50.874078, 50.873808, 50.87873],
    'lon': [4.700708, 4.700632, 4.700262, 4.700071, 4.69985, 4.70005, 4.700007, 4.70115]
}

gps_df = pd.DataFrame(gps_data)
gps_df.head(10)
  

,description,lat,lon
0,MP 01: Naamsestraat 35 Maxim,50.877121,4.700708
1,MP 02: Naamsestraat 57 Xior,50.876500,4.700632
2,MP 03: Naamsestraat 62 Taste,50.875900,4.700262
3,MP 04: His & Hears,50.875237,4.700071
4,MP 05: Calvariekapel KU Leuven,50.874520,4.699850
5,MP 06: Parkstraat 2 La Filosovia,50.874078,4.700050
6,MP 07: Naamsestraat 81,50.873808,4.700007
7,MP08bis - Vrijthof,50.878730,4.701150


In [17]:
# merge noise data with gps
merged = pd.merge(data_noise, gps_df, on='description', how='left')

# Create a StandardScaler object
scaler = MinMaxScaler()

# Fit the StandardScaler to the column and transform the lamax values
merged['standardized_lamax'] = scaler.fit_transform(merged[['lamax']])

# Fit the StandardScaler to the column and transform the lamax values
merged['standardized_laeq'] = scaler.fit_transform(merged[['laeq']])

merged.head()

,month,description,lamax,laeq,lat,lon,standardized_lamax,standardized_laeq
0,1,MP 03: Naamsestraat 62 Taste,53.239009,51.727544,50.875900,4.700262,0.656721,0.688397
1,1,MP 05: Calvariekapel KU Leuven,50.374792,48.560197,50.874520,4.699850,0.417642,0.407849
2,1,MP 06: Parkstraat 2 La Filosovia,50.086348,48.274795,50.874078,4.700050,0.393565,0.382569
3,1,MP 07: Naamsestraat 81,48.800443,47.515371,50.873808,4.700007,0.286230,0.315303
4,2,MP 01: Naamsestraat 35 Maxim,57.287668,55.245502,50.877121,4.700708,0.994666,1.000000


In [15]:
# check for missing values in each column
print(merged.isnull().sum())

month                 0
description           0
lamax                 0
laeq                  0
lat                   0
lon                   0
standardized_lamax    0
standardized_laeq     0
dtype: int64


In [19]:
# Add initial trace to the figure
fig = px.scatter_mapbox(merged, 
                        lat = 'lat', 
                        lon = 'lon', 
                        size = 'standardized_lamax',
                        size_max = 30,
                        animation_frame="month",
                        zoom = 4, mapbox_style = 'open-street-map')
# Set the initial center and zoom level of the map
fig.update_layout(mapbox={
    'center': {'lat': 50.874520, 'lon': 4.699850},
    'zoom': 15
})

In [25]:
# Add dropdown (still need to figure it out how to make the map change by clicking the button:()
fig.update_layout(
    updatemenus=[
        dict(
            type = "buttons",
            direction = "left",
            buttons=list([
                dict(
                    args=["standardized_lamax"],
                    label="Lamax",
                    method="restyle"
                ),
                dict(
                    args=["standardized_laeq"],
                    label="Laeq",
                    method="restyle"
                )
            ]),
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.0,
            xanchor="left",
            y=1.3,
            yanchor="top"
        ),
    ]
)


In [12]:
fig = px.scatter_mapbox(merged, 
                        lat = 'lat', 
                        lon = 'lon', 
                        size = 'Standardized',
                        size_max = 30,
                        animation_frame="month",
                        zoom = 4, mapbox_style = 'open-street-map')
# Set the initial center and zoom level of the map
fig.update_layout(mapbox={
    'center': {'lat': 50.874520, 'lon': 4.699850},
    'zoom': 15
})

fig.show()